Adapted from the notebook found at [How to Build a Law Bot](https://lawyerist.com/how-build-law-bot/)

## Install libraries

If you haven't already, you may need to install some dependencies. On the command line, run the following to install/update gspread, oauth2client, PyOpenSSL, and python-twitter.
```
pip install gspread
pip install --upgrade oauth2client
pip install PyOpenSSL
pip install python-twitter
```
Library installs are one and done. So after doing this once, you should be all set. 

## Import modules and set variables

Now we're getting into the bot's code. This is what will run every time your bot is called. 

You will need to create a new Google Sheet (same instructions as [last time](https://lawyerist.com/126074/online-forms-meet-local-document-automation-cut-and-paste-coding/)). Delete rows 2-999. This is because the code below appends values to the end of your sheet. So if you fail to remove rows 2-999, values will be appended to row 1000. Additionally, it looks at the last row of the sheet for your old values. So right off the bat it will be looking at your one solitary row. Also, delete columns D through Z to avoind having to print a bunch of empty columns.

As for a Twitter account and Twitter credentials, follow the instruction in [this post](https://lawyerist.com/?p=127093). 

In [28]:
# Load the module for visiting and reading websites.
import urllib.request
# Load the module for running regular expressions (regex).
import re 
# Load the module for date and time stuff.
import datetime
# Define the variable now as equal to the current date and time.
now = datetime.datetime.now()

In [29]:
# Set the URL you want to scrape.
url_1 = "http://www.aljazeera.com/topics/country/north-korea.html"

# If you want to scrape data from multiple pages, you can, 
# just replicate the above and below but change url_1 to url_2 et al.

In [30]:
# Load the module for accessing Google Sheets.
import gspread
# Load the module needed for securely communicating with Google Sheets.
from oauth2client.service_account import ServiceAccountCredentials
# The scope for your access credentials
scope = ['https://spreadsheets.google.com/feeds']

# Your spreadsheet's ID
document_key = "164Ae9PtqxwxTwO2l9y9qxtN4ZIpKdPLsjlHl5wDJOuI" 
#              ^^^^^^^^^^^ SWAP OUT FOR YOUR DOCUMENT ID/KEY
# Your Google project's .json key
credentials = ServiceAccountCredentials.from_json_keyfile_name('../../../../../SheetsBot-252f10f5aedd.json', scope)
#                                                                              ^^^^^^^^ SWAP OUT FOR YOUR JSON KEY
# Use your credentials to authorize yourself.
gc = gspread.authorize(credentials)
# Open up the Sheet with the defined ID.
wks = gc.open_by_key(document_key)

#########################################
#
#  NOTE: The name of the sheet you are 
#  trying to access should be in the 
#  parenthetical below (e.g., Data). By
#  Default this is probably "Sheet1".
#
#########################################
worksheet = wks.worksheet("Sheet1")

# Count the number of rows in your Sheet &
# resize to remove blank rows.
worksheet.resize(worksheet.row_count)

In [31]:
# Print out the old values stored in your sheet 
# Note: The first time you run this code, it will be empty as nothing has yet to be stored in your sheet.

print(worksheet.row_values(worksheet.row_count))
#############################
# DELETE CELL AFTER TESTING
#############################

['2017-10-24 12:57:58', '/programmes/insidestory/2017/10/abe-change-japan-pacifist-constitution-171023114958518.html', '', '']


In [32]:
# Import the relevant Twitter libraries so you can use Twitter.
import twitter
from twitter import TwitterError

# create the following four text files and add them to the same diretctry as you 
# Google API key. In each file add the appropriate value found when retrieving your 
# Twitter credentials

with open('../../../../../key.txt', 'r') as myfile:
    key=myfile.read()
    
with open('../../../../../secret.txt', 'r') as myfile:
    secret=myfile.read()
    
with open('../../../../../token_key.txt', 'r') as myfile:
    token_key=myfile.read()

with open('../../../../../token_secret.txt', 'r') as myfile:
    token_secret=myfile.read()

# Set you Twitter API credentials.
api = twitter.Api(consumer_key=key,
                  consumer_secret=secret,
                  access_token_key=token_key,
                  access_token_secret=token_secret)

## Read the contents of your first webpage

When you run the next cell, your program will visit the first URL you defined above. It will then print out that page's HTML. 

In [33]:
p_1 = urllib.request.build_opener(urllib.request.HTTPCookieProcessor).open(url_1).read()
print(p_1)

b'\xef\xbb\xbf\r\n<!DOCTYPE html>\r\n\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head><meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0 maximum-scale=1.0, minimal-ui" />\r\n\r\n\r\n\r\n\r\n\r\n    \n        \n\n        <!-- header tags -->\n        <meta name="guid" content="140629181426202"><title>North Korea News - Top stories from Al Jazeera</title><meta name="title" content="North Korea News - Top stories from Al Jazeera"><meta name="description" content="Stay on top of North Korea&#39;s biggest stories by Al Jazeera."><meta name="keywords" content=""><meta name="GUID" content="140629181426202"><meta name="Parent_GUID" content="131112132311520"><meta property="og:description" content=""><meta property="og:url" content="http://www.aljazeera.com/topics/country/north-korea.html"><meta name="url" content="http://www.aljazeera.com/topics/country/north-korea.html"><meta property="og:image" content="http://www.aljazeera.com"><meta name="image"

# Two Data Points, One Match

---------------------------
## Parse the site's contents

In [34]:
res_1 = re.search(b"top-feature-overlay-cont\">.*<a href=\"(.*)\">.*top-sec-title\">(.*)</h2>",p_1)
output_1 = res_1.group(1).decode('UTF-8')
print(output_1)
output_2 = res_1.group(2).decode('UTF-8')
print(output_2)

/programmes/insidestory/2017/10/abe-change-japan-pacifist-constitution-171023114958518.html
 Will Abe change Japan's pacifist constitution? 


## Post to Twitter and Save to Google (Two Data Point, One Match)

In [35]:
if (res_1 and (worksheet.row_values(worksheet.row_count)[1]) != output_1
          and (worksheet.row_values(worksheet.row_count)[2]) != output_2):
    # same as above but now comparing two values
    
    try:
        # Post to Twitter.
        status = api.PostUpdate('%s http://www.aljazeera.com/%s'%(output_2,output_1))
        print(status.text)
    except TwitterError:
        # Post to Twitter.
        status = api.PostUpdate('headline: %s http://www.aljazeera.com%s'%(output_2,output_1))
        print(status.text)

    # Save to Google only after Tweeting
    worksheet.append_row([now,output_1,output_2])